In [40]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature

In [41]:
df = pd.read_csv('../data/telecom_customer_churn_clean.csv')
df.head(20)

,Gender,Age,Married,Number of Dependents,City_0,City_1,City_2,City_3,City_4,City_5,...,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Customer Status
0,0,37,1,0,0,0,0,0,0,0,...,1,0,0,1,2,593.30,0.00,0,381.51,0
1,1,46,0,0,0,0,0,0,0,0,...,0,1,1,0,1,542.40,38.33,10,96.21,0
2,1,50,0,0,0,0,0,0,0,0,...,0,0,0,1,1,280.85,0.00,0,134.60,1
3,1,78,1,0,0,0,0,0,0,0,...,1,1,0,1,1,1237.85,0.00,0,361.66,1
4,0,75,1,0,0,0,0,0,0,0,...,1,0,0,1,1,267.40,0.00,0,22.14,1
5,0,23,0,3,0,0,0,0,0,0,...,1,1,1,1,1,571.45,0.00,0,150.93,0
6,0,67,1,0,0,0,0,0,0,0,...,1,1,1,1,3,7904.25,0.00,0,707.16,0
7,1,52,1,0,0,0,0,0,0,0,...,0,0,0,0,3,5377.80,0.00,20,816.48,0
8,0,68,0,0,0,0,0,0,0,0,...,0,0,0,1,3,340.35,0.00,0,73.71,0
9,0,43,1,1,0,0,0,0,0,0,...,1,1,1,1,3,5957.90,0.00,0,1849.90,0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589 entries, 0 to 6588
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Gender                       6589 non-null   int64  
 1   Age                          6589 non-null   int64  
 2   Married                      6589 non-null   int64  
 3   Number of Dependents         6589 non-null   int64  
 4   City_0                       6589 non-null   int64  
 5   City_1                       6589 non-null   int64  
 6   City_2                       6589 non-null   int64  
 7   City_3                       6589 non-null   int64  
 8   City_4                       6589 non-null   int64  
 9   City_5                       6589 non-null   int64  
 10  City_6                       6589 non-null   int64  
 11  City_7                       6589 non-null   int64  
 12  City_8                       6589 non-null   int64  
 13  City_9            

In [43]:
X = df.drop("Customer Status", axis=1)
y = df["Customer Status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train.shape, X_test.shape

((4612, 34), (1977, 34))

In [44]:
X.head()

,Gender,Age,Married,Number of Dependents,City_0,City_1,City_2,City_3,City_4,City_5,...,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges
0,0,37,1,0,0,0,0,0,0,0,...,1,1,0,0,1,2,593.30,0.00,0,381.51
1,1,46,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,542.40,38.33,10,96.21
2,1,50,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,280.85,0.00,0,134.60
3,1,78,1,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1237.85,0.00,0,361.66
4,0,75,1,0,0,0,0,0,0,0,...,1,1,0,0,1,1,267.40,0.00,0,22.14


# Logistic Regression 

In [46]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Depi_Log_Reg")
client = MlflowClient()

model_name = "Logistic Regression"
alias_name = "LR"

with mlflow.start_run(run_name=f"Run for @{alias_name}") as run:
    
    logreg = LogisticRegression(
        random_state=30,
        max_iter=300
    )

    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    sample_df = pd.DataFrame(X.iloc[[0]])
    signature = infer_signature(X_train, y_pred)
    mlflow.sklearn.log_model(logreg, name="model", signature=signature, input_example=sample_df)
    model_uri = f"runs:/{run.info.run_id}/model"
    result = mlflow.register_model(model_uri=model_uri, name=model_name)
    version = result.version
    client.set_registered_model_alias(
        name=model_name,
        alias=alias_name,
        version=version
    )

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Model version: {version}")
    print(f"Alias @{alias_name} → version {version}")


Registered model 'Logistic Regression' already exists. Creating a new version of this model...
2025/12/10 20:16:58 WARNING mlflow.tracking._model_registry.fluent: Run with id bf99722378434eb1b4da9b16a30074d4 has no artifacts at artifact path 'model', registering model based on models:/m-4525ada3743b4b8a8f5462e73729aadc instead
2025/12/10 20:16:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression, version 3
Created version '3' of model 'Logistic Regression'.


Accuracy: 0.8498
Precision: 0.7391
Recall: 0.7273
F1 Score: 0.7332
Model version: 3
Alias @LR → version 3
🏃 View run Run for @LR at: http://127.0.0.1:5000/#/experiments/705893408494901771/runs/bf99722378434eb1b4da9b16a30074d4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/705893408494901771


# Logistic Regression with Grid Search 

In [48]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Depi_Log_Reg")
client = MlflowClient()

with mlflow.start_run(run_name=f"Run for @{alias_name}") as run:

    model_name = "Logistic Regression with Grid Search"
    alias_name = "LR-GSCV"

    pipe_steps = [('scaler', StandardScaler()),
                  ('lr', LogisticRegression(random_state=30, class_weight="balanced"))]

    pipeline = Pipeline(pipe_steps)
    
    param_grid = {
        "lr__C": [0.1, 1, 10, 100],
        "lr__penalty": ["l2"],            
        "lr__max_iter": [200, 300 ,500,700],
    }
    
    # logreg = LogisticRegression(
    #     random_state=30,
    #     class_weight="balanced"
    #)

    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=3,
        scoring="f1_weighted",
        n_jobs=-1,
        verbose=1
    )
    
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted")
    recall = recall_score(y_test, y_pred, average="weighted")
    f1 = f1_score(y_test, y_pred, average="weighted")
    
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("best_cv_score", grid.best_score_)
    mlflow.log_params(grid.best_params_)

    sample_df = pd.DataFrame(X.iloc[[0]])
    signature = infer_signature(X_train, y_pred)
    mlflow.sklearn.log_model(logreg, name="model", signature=signature, input_example=sample_df)
    model_uri = f"runs:/{run.info.run_id}/model"
    result = mlflow.register_model(model_uri=model_uri, name=model_name)
    version = result.version

    client.set_registered_model_alias(
        name=model_name,
        alias=alias_name,
        version=version
    )

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Best CV Score (F1): {grid.best_score_:.4f}")
    print(f"Best Params: {grid.best_params_}")
    print(f"Model version: {version}")
    print(f"Alias @{alias_name} → version {version}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits


Registered model 'Logistic Regression with Grid Search' already exists. Creating a new version of this model...
2025/12/10 20:17:08 WARNING mlflow.tracking._model_registry.fluent: Run with id de1b1bf3160b4767aa018437b02a6f68 has no artifacts at artifact path 'model', registering model based on models:/m-c572e6019a0c4573b2dd3b391d1f16bc instead
2025/12/10 20:17:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression with Grid Search, version 5
Created version '5' of model 'Logistic Regression with Grid Search'.


Accuracy: 0.8154
Precision: 0.8505
Recall: 0.8154
F1 Score: 0.8228
Best CV Score (F1): 0.8193
Best Params: {'lr__C': 0.1, 'lr__max_iter': 200, 'lr__penalty': 'l2'}
Model version: 5
Alias @LR-GSCV → version 5
🏃 View run Run for @LR at: http://127.0.0.1:5000/#/experiments/705893408494901771/runs/de1b1bf3160b4767aa018437b02a6f68
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/705893408494901771
